In [ ]:
# 필요한 모듈부터 설치하기
!pip install selenium
!pip install bs4
!pip install tqdm

In [ ]:
import pandas as pd
# import 앞에 from을 붙이는 이유: 무거운 모듈에서 필요한 기능만 불러오기
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import time

In [ ]:
# 크롬창을 제어하는데 문제가 생긴다면 이 창부터 실행해야 함.
# 마이홈포털 사이트 접속하기
driver = webdriver.Chrome('chromedriver.exe')
# Mac 환경에서는 'chromedriver'로 실행해야 함.
driver.get("https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do")

In [ ]:
임대유형 = '행복주택'
# '.find_element_by_'는 필요한 요소를 찾아 크롬을 제어하기 위한 명령어임.
# link_text로 '행복주택'을 찾아서 .click()으로 클릭
driver.find_element_by_link_text(임대유형).click()
# '검색하기'의 xpath 값을 찾아 """ """(따옴표 세쌍) 안에 넣고 클릭
driver.find_element_by_xpath("""//*[@id="frm"]/div[3]/span[1]/a""").click()
# 위 코드는 driver.find_element_by_link_text('검색하기').click()와 같은 기능을 갖고 있음.
# 약 3초간 기다리기
time.sleep(3)

# 해당 페이지에 있는 임대료 정보 긁어오기

In [ ]:
# 해당 페이지의 소스를 긁어서 '소스'에 저장하고, 이를 구조화하여 soup에 저장함
소스 = driver.page_source
soup = BeautifulSoup(소스, 'html.parser')

In [ ]:
# html 소스에서 table(표) 형태의 자료를 긁어오기
목록 = pd.read_html(소스)[0]
del 목록['관심주택']
목록

In [ ]:
# table에서 필요한 열인 '단지명'을 변수에 저장하기
단지명 = 목록['단지명'].tolist()
for 단지 in 단지명:
    print(단지)

In [ ]:
# '단지명'과 일치하는 링크를 클릭하기
driver.find_element_by_link_text(단지명[0]).click()

In [ ]:
# 마지막으로 열린 창으로 전환하기
driver.switch_to.window(driver.window_handles[-1])

In [ ]:
# 새로 열린 창의 html 소스 다시 긁어오기
소스2 = driver.page_source
soup2 = BeautifulSoup(소스2, 'html.parser')

In [ ]:
# 첫 번째 테이블에서 필요한 데이터인 '준공일자'만 불러오기
# 희소성 있는 'th'태그에서 준공일자를 찾아,
# parent(부모) 구조에서 다시 'td' 태그를 찾고, 그 안에 저장된 text를 찾음.
준공일자 = soup2.find_all('th')[3].parent.find('td').text

In [ ]:
# 두 번째 table로 저장되어 있는 표를 긁어오기
#임대료 = pd.read_html(소스2)[1]
# 임대료.columns = pd.read_html(소스2)[1].columns[:-1]
# Mac의 경우 html 소스를 불러오는데 오류가 발생하여 아래와 같이 실행해야 함.
임대료 = pd.read_html(소스2)[2]
임대료.columns = pd.read_html(소스2)[1].columns[:-1]
임대료['준공일자'] = 준공일자
임대료

In [ ]:
# 목록에서 단지명과 일치하는 행만 반환하여 임대료 정보를 결합함.
# DF1.join(DF2)는 DF1(왼쪽)에 DF2(오른쪽)을 결합하는 명령어
# how = 'right'는 두 자료의 행의 길이가 다를 경우 오른쪽을 중심으로 결합하는 명령어
# .fillna(method = 'ffill')는 NaN 값을 바로 위에 있는 값으로 대체하라는 명령어
기본정보 = 목록.loc[목록['단지명']==단지명[0]].reset_index(drop=True)
통합 = 기본정보.join(임대료, how='outer').fillna(method='ffill')
통합

In [ ]:
# 현재 보고있는 창을 닫는 명령어
driver.close()
# 맨 처음 열린 창으로 전환하기
driver.switch_to.window(driver.window_handles[0])

In [ ]:
# 목록을 찾아서 임대료 정보를 긁어오는 과정을 함수로 정의하기 
def 단지정보(driver):
    소스 = driver.page_source
    soup = BeautifulSoup(소스, 'html.parser')
    목록 = pd.read_html(소스)[0]
    del 목록['관심주택']
    단지명 = 목록['단지명'].tolist()
    #함수에서는 통합을 빈 목록으로 지정 후, 루프 안의 임시저장과 통합하도록 함.
    통합 = pd.DataFrame()
    for 단지 in 단지명:
        # try: except: 구문은 오류에 대처하기 위한 명령어 
        try:
            driver.find_element_by_link_text(단지).click()
        # 단지명이 제대로 입력되지 않아, '행복주택'으로 입력된 오류에 대처하기 위한 코드
        except:
            driver.find_elements_by_link_text(단지)[3].click()
        time.sleep(3)
        driver.switch_to.window(driver.window_handles[-1])
        소스2 = driver.page_source
        soup2 = BeautifulSoup(소스2, 'html.parser')
        준공일자 = soup2.find_all('th')[3].parent.find('td').text
        임대료 = pd.read_html(소스2)[2]
        임대료.columns = pd.read_html(소스2)[1].columns[:-1]
        기본정보 = 목록.loc[목록['단지명']==단지].reset_index(drop=True)
        임시저장 = 기본정보.join(임대료, how='right').fillna(method='ffill')
        통합 = pd.concat([통합, 임시저장], sort=False)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    # '단지정보' 함수를 실행하면 통합이라는 목록을 반환하도록 함.
    return(통합)

In [ ]:
# driver를 변수로 넣어서 단지정보() 함수를 실행하여, 그 결과를 '저장'에 저장함.
저장 = 단지정보(driver)
저장 = 저장.reset_index(drop=True)

In [ ]:
저장

# 페이지 넘기는 함수 만들기

In [ ]:
소스 = driver.page_source
soup = BeautifulSoup(소스, 'html.parser')

In [ ]:
# 'div' 태그에서 'id'가 'pageDiv'인 소스를 찾기
# 찾은 소스에서 'li' tag를 가진 모든 소스를 찾기
페이지수 = soup.find('div', {'id': 'pageDiv'}).find_all('li')

In [ ]:
# 페이지수만큼 페이지를 클릭하는 함수
# 페이지 정보 소스는 //*[@id="pageDiv"]/ul/li[숫자]/a 형태로 저장되어 있음.
# .format()을 이용하여 xpath 안에서 루프의 순서에 따라 다른 숫자를 입력하도록 함.
# 페이지.text로 페이지 숫자를 텍스트(문자열)로 반환하도록 함.
for 페이지 in 페이지수:
    driver.find_element_by_xpath("""//*[@id="pageDiv"]/ul/li[{}]/a""".format(페이지.text)).click()
    time.sleep(3)

In [ ]:
def 페이지넘김(driver):
    소스 = driver.page_source
    soup = BeautifulSoup(소스, 'html.parser')
    페이지수 = soup.find('div', {'id': 'pageDiv'}).find_all('li')
    # 단지정보() 함수와 마찬가지로 통합 목록을 만들어서 반환함.
    통합 = pd.DataFrame()
    for 페이지 in 페이지수:
        driver.find_element_by_xpath("""//*[@id="pageDiv"]/ul/li[{}]/a""".format(페이지.text)).click()
        time.sleep(4)
        임시저장 = 단지정보(driver)
        통합 = pd.concat([통합, 임시저장], sort=False)
    return(통합)

In [ ]:
# 단지정보() 함수와 마찬가지로 페이지넘김() 함수의 결과를 저장함.
저장 = 페이지넘김(driver)
저장 = 저장.reset_index(drop=True)

# 광역 단위로 페이지 전환하는 함수 만들기

In [ ]:
소스 = driver.page_source
soup = BeautifulSoup(소스, 'html.parser')

In [ ]:
# '지역정보'를 저장하고 있는 'select' tag를 호명한 후,
# 'option'이 담고 있는 광역을 지역에 저장함.
광역 = soup.find('select').find_all('option')
광역

In [ ]:
# soup과 마찬가지로 driver에서도 find ~ 명령을 실행함.
# tag 이름이 'select'인 소스를 찾은 후,
# send_keys('문자열')을 이용하여 원하는 값을 입력하도록 함.
driver.find_element_by_tag_name('select').send_keys('부산광역시')

In [ ]:
# 광역 목록에서 '전체'를 제외한 2번째 값부터 루프를 시작함.
for 시도 in 광역[1:]:
    driver.find_element_by_tag_name('select').send_keys(시도.text)
    driver.find_element_by_link_text('검색하기').click()
    time.sleep(3)

In [ ]:
def 광역넘기기(driver):
    소스 = driver.page_source
    soup = BeautifulSoup(소스, 'html.parser')
    광역 = soup.find('select').find_all('option')
    통합 = pd.DataFrame()
    # tqdm_notebook()은 진행상황을 표시하는 명령어
    for 시도 in tqdm_notebook(광역[9:]):
        driver.find_element_by_tag_name('select').send_keys(시도.text)
        driver.find_element_by_link_text('검색하기').click()
        time.sleep(4)
        임시저장 = 페이지넘김(driver)
        통합 = pd.concat([통합, 임시저장], sort=False)
        # 긴 시간 작업을 진행하면서 오류가 생기는 것을 방지하여 중간중간 광역별로 저장하도록 함.
        광역 = 통합.loc[통합['주소'].str.contains(시도.text)]
        광역.to_csv('마이홈포털_20191119_행복주택_{}.csv'.format(시도.text), index=False)
    return(통합)

In [ ]:
저장 = 광역넘기기(driver)

In [ ]:
# 오류가 생길 우려가 있으므로 다시 광역별 파일을 불러와서 통합하기
통합 = pd.DataFrame()
for 시도 in 광역[1:]:
    임시저장 = pd.read_csv('마이홈포털_20191119_행복주택_{}.csv'.format(시도.text),
                      engine='python', encoding='utf-8')
    통합 = pd.concat([통합, 임시저장], sort=False)
del 임시저장

In [ ]:
# 통합된 데이터를 정제하기
# 통합 = 저장.copy()로 오류없이 실행된 파일을 불러올 수도 있음
통합['공급면적(공용)'] = 통합['공급면적(공용)'].str.replace('㎡', '')
통합['공급면적(전용)'] = 통합['공급면적(전용)'].str.replace('㎡', '')
통합['임대보증금'] = 통합['임대보증금'].str.replace('원', '')
통합['임대보증금'] = 통합['임대보증금'].str.replace(',', '')
통합['임대료'] = 통합['임대료'].str.replace('원', '')
통합['임대료'] = 통합['임대료'].str.replace(',', '')
통합['전환보증금'] = 통합['전환보증금'].str.replace('원', '')
통합['전환보증금'] = 통합['전환보증금'].str.replace(',', '')

In [ ]:
통합.to_csv('마이홈포털_20191119_행복주택.csv', index=False)